In [8]:
from agents import *
from random import randrange, seed, choice

# A Mining * is Born!
This program simulates mining for gold. Unlike the previous assignment, this time the robot can actually sense what is around it! We will set the perceptible distance to 2, giving the robot knowledge of a small slice of its grid world. One thing to keep in mind is that everything the robot sees will be a Thing, and things have "location" variables which indicate their exact coordinate.

In order to help our little buddy out, we are going to code an A* search algorithm to get it onto desired squares.

## The Environment
First, we define a Gold class as well as a graphical environment for the robot to explore, we then generate a random mine. Read over this code, but there is nothing that needs changing here. The environment is drawn as a grid. Black squares are walls, which the robot cannot pass through. The gold squares are squares containing gold and the grey squares are empty squares. Finally, when we add the robot, it will show up as a red square. Perciptible distance is set to 2.

In [9]:
class Gold(Thing):
    """
    A chunk of gold, it does nothing! (But the Wumpus gold in agents is a singleton)
    so we need this one here.
    """
    pass

class MineEnvironment(GraphicEnvironment):
    def __init__(self):
        color = {"Miner": (255,0,0),
                 "Gold": (253, 208, 23),
                 "Wall": (44, 53, 57)}
        super().__init__(16, 16, True, color)
        self.perceptible_distance=2
        self.add_walls()

        # place obstacles
        for i in range(64):
            self.add_thing(Wall(), self.empty_square())
        
        # place gold
        for i in range(42):
            self.add_thing(Gold(), self.empty_square())
    
    def empty_square(self):
        world = self.get_world()
        done = False
        while not done:
            x = randrange(len(world))
            y = randrange(len(world[x]))
            done = len(world[x][y]) == 0
        return x,y

mine = MineEnvironment()
mine.reveal()

,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


## The Star Miner
Now we come to the star of the show! Our intrepid little miner robot. The robot enters the field on a random empty square. Its constructor sets up the robots program and sets its initial direction to facing to the right. The robot can perform the following operations:

- TurnLeft
- TurnRight
- Forward
- Grab

These actions are perfomed by returning them from the program function. The program function receives an array of percepts, which is an array of tuples consisting of the objects on the robot's current square, as well as its **squared straight line distance** from the robot. A sample array is shown below.

```
[(<Gold>, 4), (<Wall>, 1), (<Wall>, 2), (<Miner>, 0)]
```

The above array would mean that there is a gold brick 2 units away, a wall 1 unit away, a wall sqrt(2) units away and miner 0 units away (the miner will always be there.).

In addition to the percept sequence, there is also the variable `self.bump` which is true if the robot rolled into a wall on its last move.

Note how the Miner is always included in the percept list. This is because the miner is part of the environment too! If you attempt to "Grab", the environment makes sure this is something that can be grabbed by calling the "can_grab" function.

In [10]:
class Miner(Agent):
    def __init__(self, program=None):
        super().__init__(self.program)
        self.direction = Direction("right")
        self.history=[]
        self.plan=[]
        
    def can_grab(self, thing):
        return thing.__class__.__name__ == "Gold"
    
    def program(self, percept):
        #TODO: Do something better rolling in our Konami path
        if len(self.plan) == 0:
            self.plan = [self.up, self.up, self.down, self.down, self.left, self.right, self.left, self.right]
        
        # execute the next part of the plan
        return self.plan[0]()
    
    def up(self):
        """
        Execute the next step in a plan to go up. When the robot gloriously completes 
        its mission, it pops the action from the sequence.
        """
        if self.direction.direction == Direction.U:
            self.plan.pop(0)
            return "Forward"
        elif self.direction.direction == Direction.L:
            return "TurnRight"
        else:
            return "TurnLeft"
    
    def down(self):
        """
        Execute the next step in a plan to go down. When the robot gloriously completes 
        its mission, it pops the action from the sequence.
        """
        if self.direction.direction == Direction.D:
            self.plan.pop(0)
            return "Forward"
        elif self.direction.direction == Direction.R:
            return "TurnRight"
        else:
            return "TurnLeft"
    
    def left(self):
        """
        Execute the next step in a plan to go left. When the robot gloriously completes 
        its mission, it pops the action from the sequence.
        """
        if self.direction.direction == Direction.L:
            self.plan.pop(0)
            return "Forward"
        elif self.direction.direction == Direction.D:
            return "TurnRight"
        else:
            return "TurnLeft"
    
    def right(self):
        """
        Execute the next step in a plan to go up. When the robot gloriously completes 
        its mission, it pops the action from the sequence.
        """
        if self.direction.direction == Direction.R:
            self.plan.pop(0)
            return "Forward"
        elif self.direction.direction == Direction.U:
            return "TurnRight"
        else:
            return "TurnLeft"
        

miner = Miner()
mine.add_thing(miner, mine.empty_square())
mine.run(50)

,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


## How did we do?
The code below displays the robot's final amount of collected gold.

In [11]:
print("Gold Collected: {}".format(len(miner.holding)))


Gold Collected: 0
